In [ ]:
import torch.nn.functional as F

class SimpleEncoder(nn.Module):
    """
    Basit CNN Encoder - 3x32x32 -> 256
    32x32 -> 16x16 -> 8x8 -> 4x4 -> 2x2 -> 256
    """
    def __init__(self, input_channels=3, latent_dim=256):
        super(SimpleEncoder, self).__init__()
        
        # İlk katman: 3x256x256 -> 32x256x256
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        # Down sampling 1: 32x256x256 -> 64x128x128
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # Down sampling 2: 64x128x128 -> 128x64x64
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Down sampling 3: 128x64x64 -> 256x32x32
        self.conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Down sampling 4: 256x32x32 -> 512x16x16
        self.conv5 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        
        # Final: 2048x4x4 -> 256
        self.flatten_size = 512 * 4 * 4
        
        # Son katman: 2048 -> 256 (sigmoid aktivasyonu ile)
        # tayfun self.fc = nn.Linear(self.flatten_size, latent_dim)

        self.fc = nn.Sequential(
            nn.Linear(self.flatten_size, 2048),
            nn.SiLU(),
            nn.BatchNorm1d(2048),
            
            nn.Linear(2048, latent_dim),
            nn.Sigmoid()
        )
    def forward(self, x):
        # Adım adım down sampling
        # Input: 3x256x256
        x = self.bn1(F.silu(self.conv1(x)))    # -> 32x32x32
        x = self.bn2(F.silu(self.conv2(x)))    # -> 64x16x16
        x = self.bn3(F.silu(self.conv3(x)))    # -> 128x8x8
        x = self.bn4(F.silu(self.conv4(x)))    # -> 256x4x4
        x = self.bn5(F.silu(self.conv5(x)))    # -> 512x2x2
        
        # Flatten
        x = x.view(x.size(0), -1)  # -> batch_size x 2048
        
        # Final layer with sigmoid activation
        # tayfun encoded = torch.sigmoid(self.fc(x))    # -> batch_size x 256 (0-1 arası)
        encoded = self.fc(x)
        
        return encoded

class SimpleDecoder(nn.Module):
    """
    Basit CNN Decoder - 256 -> 3x32x32
    256 -> 2x2 -> 4x4 -> 8x8 -> 16x16 -> 32x32
    """
    def __init__(self, latent_dim=256, output_channels=3):
        super(SimpleDecoder, self).__init__()
        
        # Latent'i feature map'e çevir: 256 -> 512x2x2
        # tayfun self.fc = nn.Linear(latent_dim, 512 * 2 * 2)
        self.fc = nn.Sequential(
            nn.Linear(latent_dim,  2048),
            nn.SiLU(),
            
            nn.Linear(2048, 512 * 4 * 4)
        )
 
        # Up sampling 1: 512x16x16 -> 256x32x32
        self.deconv1 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(256)
        
        # Up sampling 2: 256x32x32 -> 128x64x64
        self.deconv2 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        # Up sampling 3: 128x64x64 -> 64x128x128
        self.deconv3 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        # Up sampling 4: 64x128x128 -> 32x256x256
        self.deconv4 = nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        
        # Final output: 32x256x256 -> 3x256x256 (sigmoid aktivasyonu)
        self.final_conv = nn.Conv2d(32, output_channels, kernel_size=3, padding=1)
        
    def forward(self, z):
        # Latent'i feature map'e çevir
        x = self.fc(z)                         # 256 -> 2048
        x = x.view(x.size(0), 512 , 4 , 4)      # -> 512x2x2
        
        # Adım adım up sampling
        x = F.silu(self.bn1(self.deconv1(x)))  # -> 256x4x4
        x = F.silu(self.bn2(self.deconv2(x)))  # -> 128x8x8
        x = F.silu(self.bn3(self.deconv3(x)))  # -> 64x16x16
        x = F.silu(self.bn4(self.deconv4(x)))  # -> 32x32x32
        
        # Final output with sigmoid activation
        x = torch.sigmoid(self.final_conv(x))  # -> 3x32x32 ([0, 1] aralığında)
        
        return x

class To_Uniform(nn.Module):
    """
    Basit CNN Autoencoder - 3x32x32 <-> 256 latent dim
    """
    def __init__(self, input_channels=3, latent_dim=256, output_channels=3):
        super(To_Uniform, self).__init__()
        
        self.encoder = SimpleEncoder(input_channels, latent_dim)
        self.decoder = SimpleDecoder(latent_dim, output_channels)
        self.latent_dim = latent_dim
    
    def forward(self, x):
        # Encode: 3x32x32 -> 256 (sigmoid ile 0-1 arası)
        encoded = self.encoder(x)
        
        # Decode: 256 -> 3x32x32 (sigmoid ile 0-1 arası)
        reconstructed = self.decoder(encoded)
        
        return encoded, reconstructed
    
    def encode(self, x):
        """Sadece encode et"""
        return self.encoder(x)
    
    def decode(self, z):
        """Sadece decode et"""
        return self.decoder(z)

    def criterion(self, z_pred, z_true, x_pred, x_true):
        return -1.0 * F.mse_loss(z_pred, z_true) + 1.0 * F.mse_loss(x_pred, x_true)